In [1]:
%load_ext autoreload
%autoreload 2

# How to install

I would install the module as a git submodule in your project folder with:

`git submodule add git@github.com:TimonKunze/CqaSimulator.git submodules/cqasim`

Then one can easily push changes to one's own project git repository as well as pushing bugfixes and extensions to the `sqasim`package itself. 

I would however mostly work without changing the module but only by inheriting from the module classes and overwriting the methods...

**Comment_1 to Tadeo and Aiswarya: If we want to change the module it's maybe also best to do it on a different branch first and only merge into the project after intensive testing. But I'm also new and unexperienced how to do this. I never built such a package before or worked with it collaboratively from version 0.0. Let me know how you think it's best to handle.**

**Comment_2: Although I tried to test many functions (see folder `test`, easiest is to execute these tests with `pytest test_file.py` in the terminal), I'm sure there are still several errors in it that we have to fix. Also the package could still be extended considerably, neither I implemented all of FS's measures already. Time will tell...**



In one's project root one can install the package as editable in with `pip install -e ./submodules/sqasim`.

Then one can load the load submodule like here:

```
from submodules.cqasim.cqa_simulator import CqaSimulator
from submodules.cqasim.cqa_defaults import CqaDefaults
```

and can inherit from the module classes like so:

```
class MyOwnCqaSimulator(CqaSimulator):
    def __init__(self):
        super().__init__()

    def run(self):
        ...
```



Of course there are also many other ways of how to install the code...

# How to use

In [2]:
from cqasim.cqa_simulator import *
from cqasim.cqa_defaults import *

Numba is using 8 threads.


## Get Default Parameters

Model default paramters are stored in a dictionary in `cqa_default.py`. They can be assessed as so:

In [3]:
defaults = CqaDefaults()
print(defaults)

CqaModel(par={'P': 1, 'N': 999, 'T': 1000, 'L': 200, 'mean_diameter': 1.57, 'var_diameter': 2.0, 'correlated_peaks': False, 'simplified_data': True, 'correlated_dimensions': 'min', 'mean_height': 1.549, 'var_height': 0.0, 'exponent': 0, 'gamma': 0.5, 'M': 1, 'M_fixed': True, 'seed': 1, 'track_dynamics_flag': False, 'record_final_flag': True, 'g': 17.0, 'kb': 300, 'initial_step_size': 0.08, 'converge_eps': 1e-05, 'max_iterations': 60000.0, 'verbose': False})


This dictionary can be changed when instantiating the simulator but if a model parameter is not specified, a default parameter from `cqa_default.py` is used:

In [4]:
cqa = CqaSimulator({
    "var_diameter": 0.1,
    "var_heights": 0.1,
    "N": 3000,
    "M": 4.4,
    "correlated_peaks": True,
    "M_fixed": False,
    "seed": 1,
})

Parameters can also be changed after the initialization but in this case it is important to run `cqa.update_ctx_state()` after changing the parameters. Like so:

In [ ]:
cqa = CqaSimulator()
cqa.par["var_diameter"] = 1.3
cqa.par["N"] = 6000
cqa.par["M"] = 4.4
cqa.par["correlated_peaks"] = True
cqa.par["M_fixed"] = False
cqa.__init__(cqa.par)  # This is important!!
cqa.update_ctx_state()  # This is important!!


## Run the simulator

Now one can run the simulator...

### ... until convergence for a specific poisition

In [ ]:
%matplotlib inline
print(cqa)
init_pos = 100
cqa.run_until_convergence(
    init_pos, p=0, record_final_flag=False, verbose=2,
    visualize=False,  # Visualization works better in terminal than in notebook... for now set to False
)

CqaSimulator(params={'P': 1, 'N': 6000, 'T': 1000, 'L': 200, 'mean_diameter': 1.57, 'var_diameter': 1.3, 'correlated_peaks': True, 'simplified_data': True, 'correlated_dimensions': 'min', 'mean_height': 1.549, 'var_height': 0.0, 'exponent': 0, 'gamma': 0.5, 'M': 4.4, 'M_fixed': False, 'seed': 1, 'track_dynamics_flag': False, 'record_final_flag': True, 'g': 17.0, 'kb': 300, 'initial_step_size': 0.08, 'converge_eps': 1e-05, 'max_iterations': 60000.0, 'verbose': False})
Model converged after 228 steps.


### ... until convergence for some/all positions in T

In [ ]:
print(cqa)
spacing = 200  # run for every 100th position in T
cqa.run_until_convergence_over_positions(
    spacing=spacing, record_final_flag=False, verbose=2,
)

CqaSimulator(params={'P': 1, 'N': 6000, 'T': 1000, 'L': 200, 'mean_diameter': 1.57, 'var_diameter': 1.3, 'correlated_peaks': True, 'simplified_data': True, 'correlated_dimensions': 'min', 'mean_height': 1.549, 'var_height': 0.0, 'exponent': 0, 'gamma': 0.5, 'M': 4.4, 'M_fixed': False, 'seed': 1, 'track_dynamics_flag': False, 'record_final_flag': False, 'g': 17.0, 'kb': 300, 'initial_step_size': 0.08, 'converge_eps': 1e-05, 'max_iterations': 60000.0, 'verbose': False})
p = 0 , init_pos = 0  (var_dia = 1.3 )
Model converged after 225 steps.
p = 0 , init_pos = 200  (var_dia = 1.3 )
Model converged after 227 steps.
p = 0 , init_pos = 400  (var_dia = 1.3 )
Model converged after 215 steps.
p = 0 , init_pos = 600  (var_dia = 1.3 )
Model converged after 219 steps.
p = 0 , init_pos = 800  (var_dia = 1.3 )
Model converged after 215 steps.


## Saving simulation runs

Simulation runs can be saved with `record_final_flag=True` in a data structure that contains all the parameters of the model and is specified in `default.py`. 

Some parameters that are less often changed and are related to the simulation or size of the network are included in folders names of the nested folder structure:

In [ ]:
print(defaults)

defaults.build_fp(".")

CqaModel(par={'P': 1, 'N': 999, 'T': 1000, 'L': 200, 'mean_diameter': 1.57, 'var_diameter': 2.0, 'correlated_peaks': False, 'simplified_data': True, 'correlated_dimensions': 'min', 'mean_height': 1.549, 'var_height': 0.0, 'exponent': 0, 'gamma': 0.5, 'M': 1, 'M_fixed': True, 'seed': 1, 'track_dynamics_flag': False, 'record_final_flag': True, 'g': 17.0, 'kb': 300, 'initial_step_size': 0.08, 'converge_eps': 1e-05, 'max_iterations': 60000.0, 'verbose': False})


PosixPath('simpl_data/g17.0_kb300_step0.08/P1_N999/T1000_L200')

Others are used in the filename:

In [ ]:
defaults.build_fn()

'fieldM1_heightM1.549_diaM1.57_diaVar2.0_corrP0_gamma0.5_seed1'

To run the model over multiple parameters, for example, and save the state of the network when it converged, it is best to just specify your path `fp="..."` and call the method `cqa.save_run_data(fp)` with the `record_final_flag == True` which is default.

In such a run, the model saves individual `.npy` files of:
- "data_{fn}.npy" : The data with which the model was initialized
- "diams_per_nrn_{fn}.npy" : The diameters of place fields for each neuron
- "heights_per_nrn_{fn}.npy" : The heights of place fields for each neuron
- "fields_per_nrn_{fn}.npy" : The position of place fields for each neuron
- "sim_data_{fn}.npy" : The simualted data after the model has converged
- "sim_pos_{fn}.npy" The position of maximal overlap when the model has converged

One can also save the full model with `cqa.save_full_model(fp)` and reload it some time later with `cqa.load_full_model(fp)`.

In [ ]:
fp = "..."  # Here goes your path!
cqa.par["record_final_flag"] = True  # Should be default anyways...
cqa.update_ctx_state()

# var_dias = np.round(np.arange(0.0, 2.0, 0.05), 2).tolist()  # diameter variations
var_dias = [0.5]
for i, var_d in enumerate(var_dias):
    print(f"\n Run {i}/{len(var_dias)} with diameter variance {var_d}")

    cqa.par["var_diameter"] = var_d
    cqa.update_ctx_state()

    cqa.run_until_convergence_over_positions(spacing=500)
    # cqa.save_run_data(fp)  # Uncomment to save!
# cqa.save_full_model(fp)  # Uncomment to save!
# cqa.load_full_model(fp)  # Uncomment to save!


 Run 0/1 with diameter variance 0.5
p = 0 , init_pos = 0  (var_dia = 0.5 )
Model converged after 708 steps.
p = 0 , init_pos = 500  (var_dia = 0.5 )
Model converged after 1300 steps.


To save information about the dynamics, one should set `track_dynamics_flag == True`, and the model calculates some summary information about the dynamics at each time step. It does not, however, save all activations at every timestep since it would create to large data sets. (Not that track_dynamics slows the simulation down by quite a bit.)

So far this information includes the following (but might still be extended with other measures).
- overlap_max_{fn}.npy : the maximal overlap during each time step
- overlap_max_pos_{fn}.npy : the position of maximal overlap during each time step
- overlap_disp_{fn}.npy : the dispersion of overlap at each time step
- overlap_disp_clip_{fn}.np : the dispersion of clipped overlap at each time step
- v_disp_{fn}.npy : the dispersion of the activity V at each time step

In [ ]:
fp = "..."  # Here goes your path!
cqa.par["record_final_flag"] = True  # Should be default anyways...
cqa.par["track_dynamics_flag"] = True  # Should be default anyways...
cqa.__init__(cqa.par)  # This is important!!
cqa.update_ctx_state() # This is important!!

# var_dias = np.round(np.arange(0.0, 2.0, 0.05), 2).tolist()  # diameter variations
var_dias = [0.5]
for i, var_d in enumerate(var_dias):
    print(f"\n Run {i}/{len(var_dias)} with diameter variance {var_d}")

    cqa.par["var_diameter"] = var_d
    cqa.update_ctx_state()

    cqa.run_until_convergence_over_positions(spacing=500)
    # cqa.save_run_data(fp)  # Uncomment to save!
# cqa.save_full_model(fp)  # Uncomment to save!
# cqa.load_full_model(fp)  # Uncomment to save!


 Run 0/1 with diameter variance 0.5
p = 0 , init_pos = 0  (var_dia = 0.5 )
Model converged after 708 steps.
p = 0 , init_pos = 500  (var_dia = 0.5 )
Model converged after 1300 steps.
